In [1]:
!pip install underthesea
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 50.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 53.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.4/312.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 66.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 6.3 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=9682a79333f1e4708ba20ec4907f036afbce7f4ce2bb75e5b247bdd0363feeb9
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Unins

In [2]:
import numpy as np
import pandas as pd
from string import punctuation
import torch
from torch.utils.data import TensorDataset, DataLoader
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import underthesea

In [3]:
#preprocess.py
import regex as re
from underthesea import word_tokenize
import string

# remove html tags
def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)

# unicode stardard
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

# diacritic standard
vowels_to_ids = {}
vowels_table = [
    ['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a' ],
    ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
    ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
    ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e' ],
    ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
    ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i' ],
    ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o' ],
    ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
    ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
    ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u' ],
    ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
    ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y' ]
]

for i in range(len(vowels_table)):
    for j in range(len(vowels_table[i]) - 1):
        vowels_to_ids[vowels_table[i][j]] = (i, j)

def is_valid_vietnamese_word(word):
    chars = list(word)
    vowel_indexes = -1
    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x != -1:
            if vowel_indexes == -1: vowel_indexes = index
            else:
                if index - vowel_indexes != 1: return False
                vowel_indexes = index
    return True

def standardize_word_typing(word):
    if not is_valid_vietnamese_word(word): return word
    chars = list(word)
    dau_cau = 0
    vowel_indexes = []
    qu_or_gi = False

    for index, char in enumerate(chars):
        x, y = vowels_to_ids.get(char, (-1, -1))
        if x == -1: continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True

        if y != 0:
            dau_cau = y
            chars[index] = vowels_table[x][0]

        if not qu_or_gi or index != 1:
            vowel_indexes.append(index)

    if len(vowel_indexes) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = vowels_to_ids.get(chars[1])
                chars[1] = vowels_table[x][dau_cau]
            else:
                x, y = vowels_to_ids.get(chars[2], (-1, -1))
                if x != -1: chars[2] = vowels_table[x][dau_cau]
                else: chars[1] = vowels_table[5][dau_cau] if chars[1] == 'i' else vowels_table[9][dau_cau]
            return ''.join(chars)
        return word

    for index in vowel_indexes:
        x, y = vowels_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = vowels_table[x][dau_cau]
            return ''.join(chars)

    if len(vowel_indexes) == 2:
        if vowel_indexes[-1] == len(chars) - 1:
            x, y = vowels_to_ids[chars[vowel_indexes[0]]]
            chars[vowel_indexes[0]] = vowels_table[x][dau_cau]
        else:
            x, y = vowels_to_ids[chars[vowel_indexes[1]]]
            chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    else:
        x, y = vowels_to_ids[chars[vowel_indexes[1]]]
        chars[vowel_indexes[1]] = vowels_table[x][dau_cau]
    return ''.join(chars)

def standardize_sentence_typing(text):
    words = text.lower().split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        if len(cw) == 3: cw[1] = standardize_word_typing(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

# remove unnecessary characters
def remove_unnecessary(text):
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬÉÈẺẼẸÊẾỀỂỄỆÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÍÌỈĨỊÚÙỦŨỤƯỨỪỬỮỰÝỲỶỸỴĐ_]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra whitespace
    return text

# special wordings in reviews (e.g., emoji,...)
def normalize_money(sent):
    return re.sub(r'[0-9]+[.,0-9][k-m-b]', 'giá', sent)

def normalize_hastag(sent):
    return re.sub(r'#+\w+', 'tag', sent)

def normalize_website(sent):
    result = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', 'website', sent)
    return re.sub(r'\w+(\.(com|vn|me))+((\/+([\.\w\_\-]+)?)+)?', 'website', result)

def nomalize_emoji(sent):
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
    "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', sent)

def normalize_elongate(sent):
    patern = r'(.)\1{1,}'
    result = sent
    while(re.search(patern, result) != None):
        repeat_char = re.search(patern, result)
        result = result.replace(repeat_char[0], repeat_char[1])
    return result

def remove_numbers(sent):
    return re.sub(r'[0-9]+', '', sent)

def normalize_acronyms(sent):
    text = sent
    replace_list = {
        'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
        'okey': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
        '⭐': 'star ', '*': 'star ', '🌟': 'star ', '🎉': u' tích cực ',
        'kg ': u' không ','not': u' không ', u' kg ': u' không ', '"k ': u' không ',' kh ':u' không ','kô':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', '"ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',
        'he he': ' tích cực ','hehe': ' tích cực ','hihi': ' tích cực ', 'haha': ' tích cực ', 'hjhj': ' tích cực ',
        ' lol ': ' tiêu cực ',' cc ': ' tiêu cực ','cute': u' dễ thương ','huhu': ' tiêu cực ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',
        ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
        'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' tích cực ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
        'dep': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ',
        'iu': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' tích cực ',
        ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ',
        'ô kêi': 'ok', 'okie': 'ok', 'o kê': 'ok', 'okey': 'ok', 'ôkê': 'ok', 'oki': 'ok', 'oke': 'ok', 'okay': 'ok', 'okê': 'ok',
        'tks': 'cảm ơn', 'thks': 'cảm ơn', 'thanks': 'cảm ơn', 'ths': 'cảm ơn', 'thank': 'cảm ơn',
        'kg': 'không', 'not': 'không', 'k': 'không', 'kh': 'không', 'kô': 'không', 'hok': 'không', 'ko': 'không', 'khong': 'không', 'kp': 'không phải',
        'he he': 'tích cực', 'hehe': 'tích cực', 'hihi': 'tích cực', 'haha': 'tích cực', 'hjhj': 'tích cực', 'thick': 'tích cực',
        'lol': 'tiêu cực', 'cc': 'tiêu cực', 'huhu': 'tiêu cực', 'cute': 'dễ thương',
        
        'sz': 'cỡ', 'size': 'cỡ', 
        'wa': 'quá', 'wá': 'quá', 'qá': 'quá', 
        'đx': 'được', 'dk': 'được', 'dc': 'được', 'đk': 'được', 'đc': 'được', 
        'vs': 'với', 'j': 'gì', '“': ' ', 'time': 'thời gian', 'm': 'mình', 'mik': 'mình', 'r': 'rồi', 'bjo': 'bao giờ', 'very': 'rất',

        'authentic': 'chuẩn chính hãng', 'aut': 'chuẩn chính hãng', 'auth': 'chuẩn chính hãng', 'date': 'hạn sử dụng', 'hsd': 'hạn sử dụng', 
        'store': 'cửa hàng', 'sop': 'cửa hàng', 'shopE': 'cửa hàng', 'shop': 'cửa hàng', 
        'sp': 'sản phẩm', 'product': 'sản phẩm', 'hàg': 'hàng', 
        'ship': 'giao hàng', 'delivery': 'giao hàng', 'síp': 'giao hàng', 'order': 'đặt hàng',

        'gud': 'tốt', 'wel done': 'tốt', 'good': 'tốt', 'gút': 'tốt', 'tot': 'tốt', 'nice': 'tốt', 'perfect': 'rất tốt', 
        'quality': 'chất lượng', 'chất lg': 'chất lượng', 'chat': 'chất', 'excelent': 'hoàn hảo', 'bt': 'bình thường',
        'sad': 'tệ', 'por': 'tệ', 'poor': 'tệ', 'bad': 'tệ', 
        'beautiful': 'đẹp tuyệt vời', 'dep': 'đẹp', 
        'xau': 'xấu', 'sấu': 'xấu', 
        
        'thik': 'thích', 'iu': 'yêu', 'fake': 'giả mạo', 
        'quickly': 'nhanh', 'quick': 'nhanh', 'fast': 'nhanh',
        'fresh': 'tươi', 'delicious': 'ngon',

        'dt': 'điện thoại', 'fb': 'facebook', 'face': 'facebook', 'ks': 'khách sạn', 'nv': 'nhân viên',
        'nt': 'nhắn tin', 'ib': 'nhắn tin', 'tl': 'trả lời', 'trl': 'trả lời', 'rep': 'trả lời',
        'fback': 'feedback', 'fedback': 'feedback',
        'sd': 'sử dụng', 'sài': 'xài', 

        '^_^': 'tích cực', ':)': 'tích cực', ':(': 'tiêu cực',
        '❤️': 'tích cực', '👍': 'tích cực', '🎉': 'tích cực', '😀': 'tích cực', '😍': 'tích cực', '😂': 'tích cực', '🤗': 'tích cực', '😙': 'tích cực', '🙂': 'tích cực', 
        '😔': 'tiêu cực', '😓': 'tiêu cực', 
        '⭐': 'star', '*': 'star', '🌟': 'star',
    }
    for k, v in replace_list.items():
        text = text.replace(k, v)
    return text


# overall preprocessing
def text_preprocess(document):
    #đưa về lower
    document = document.lower()
    # xóa html code
    document = remove_html(document)
    # chuẩn hóa unicode
    document = convert_unicode(document)
    
    # chuẩn hóa các ký tự đặc biệt
    document = normalize_money(document)
    document = normalize_hastag(document)
    document = normalize_website(document)
    document = nomalize_emoji(document)
    document = normalize_elongate(document)
    document = normalize_acronyms(document)
    document = remove_numbers(document)

    # chuẩn hóa cách gõ dấu tiếng việt
    document = standardize_sentence_typing(document)
    # tách từ
    document = word_tokenize(document, format="text")
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = remove_unnecessary(document)
    return document.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).replace(' '*4, ' ').replace(' '*3, ' ').replace(' '*2, ' ').strip()

In [4]:
#vectorizer.py
from gensim.models import KeyedVectors

def tokenize_all_reviews(embed_model, dims, reviews):
    # split each review into a list of words
    reviews_words = [review.split() for review in reviews]

    tokenized_reviews = []
    for review in reviews_words:
        mean_embedding = np.zeros(dims)
        num_words = 0
        for word in review:
            try:
                mean_embedding += embed_model[word]
                num_words += 1
            except:
                continue
        mean_embedding /= num_words
        tokenized_reviews.append(mean_embedding)

    return tokenized_reviews 

def tokenize_reviews(embed_model, dims, sent):
    words = sent.split()
    mean_embedding = np.zeros(dims)
    num_words = 0
    for word in words:
        try:
            mean_embedding += embed_model[word]
            num_words += 1
        except:
            continue
    mean_embedding /= num_words
    return mean_embedding

class W2VLoader():
    def __init__(self, w2v_path=''):
        self.embed_model = KeyedVectors.load_word2vec_format(w2v_path, binary=False)
        self.pretrained_words = self.get_pretrained_words()
        self.vocab_size = len(self.pretrained_words)
        self.embedding_dim = self.get_embedding_dim()

    def get_pretrained_words(self):
        # Store pretrained vocab
        pretrained_words = list(self.embed_model.key_to_index.keys())
        return pretrained_words

    def get_embedding_dim(self):
        # Get dimensionality of embeddings
        word = self.pretrained_words[0]
        embedding_dim = self.embed_model.get_vector(word).shape[0]
        return embedding_dim


In [5]:
#data_loader.py
def clean_n_tokenize_data(sent, w2v):
    sent = text_preprocess(sent)
    tokenized_sent = tokenize_reviews(w2v.embed_model, w2v.embedding_dim, sent)
    return tokenized_sent

def read_data_from_csv():
    train = pd.read_csv('/kaggle/input/absa-phone-vi/datasets/Train.csv')
    val = pd.read_csv('/kaggle/input/absa-phone-vi/datasets/Val.csv')
    test = pd.read_csv('/kaggle/input/absa-phone-vi/datasets/Test.csv')
    return train, val, test

def load_data(data, w2v):
    data.loc[:, 'Comments'] = data['Comments'].apply(lambda x: text_preprocess(x))
    tokenized_reviews = tokenize_all_reviews(w2v.embed_model, w2v.embedding_dim, data['Comments'])

    return np.stack(pd.Series(tokenized_reviews)), data[['Pin', 'Service', 'General', 'Others', 'SPin', 'SSer', 'SGeneral', 'SOth']]

In [6]:
w2v_300dims = W2VLoader(
        w2v_path='/kaggle/input/phow2v/word2vec_vi_words_300dims/word2vec_vi_words_300dims.txt'
    )

In [7]:
train, val, test = read_data_from_csv()

X_train, y_train = load_data(train, w2v_300dims)
X_val, y_val = load_data(val, w2v_300dims)
X_test, y_test = load_data(test, w2v_300dims)

In [186]:
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

class SVCModel():

    def __init__(self, kernel='rbf', C=1.0, gamma='scale', attribute=None):
        self.kernel = kernel
        self.C = C
        self.gamma = gamma
        self.svc = None
        self.attribute = attribute

    def fit(self, X, y):
        self.svc = SVC(kernel=self.kernel, C=self.C, gamma=self.gamma, probability=True)
        self.svc.fit(X, y[self.attribute])

    def predict(self, X):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")
        return self.svc.predict(X)

    def predict_proba(self, X):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")
        return self.svc.predict_proba(X)
    
    def calculate_accuracy_score(self, X, y, y_pred=None):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")
        
        if y_pred is not None:
            accuracy = accuracy_score(y[self.attribute], y_pred)
            return accuracy
        
        y_pred = self.predict(X)
        accuracy = accuracy_score(y[self.attribute], y_pred)
        return accuracy
    
    def calculate_f1_score(self, X, y, y_pred=None):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")

        if y_pred is not None:
            if y[self.attribute].nunique() < 3:
                f1 = f1_score(y[self.attribute], y_pred)
            else:
                f1 = f1_score(y[self.attribute], y_pred, average='micro')
            return f1

        y_pred = self.predict(X)
        if y[self.attribute].nunique() < 3:
            f1 = f1_score(y[self.attribute], y_pred)
        else:
            f1 = f1_score(y[self.attribute], y_pred, average='micro')
        return f1
    
    def calculate_precision_score(self, X, y, y_pred=None):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")
        
        if y_pred is not None:
            if y[self.attribute].nunique() < 3:
                precision = precision_score(y[self.attribute], y_pred)
            else:
                precision = precision_score(y[self.attribute], y_pred, average='micro')
            return precision
        
        y_pred = self.predict(X)
        if y[self.attribute].nunique() < 3:
            precision = precision_score(y[self.attribute], y_pred)
        else:
            precision = precision_score(y[self.attribute], y_pred, average='micro')
        return precision
    
    def calculate_recall_score(self, X, y, y_pred=None):
        if self.svc is None:
            raise ValueError("SVC model has not been trained yet. Please call 'fit' first.")
        
        if y_pred is not None:
            if y[self.attribute].nunique() < 3:
                recall = recall_score(y[self.attribute], y_pred)
            else:
                recall = recall_score(y[self.attribute], y_pred, average='micro')
            return recall
        
        y_pred = self.predict(X)
        if y[self.attribute].nunique() < 3:
            recall = recall_score(y[self.attribute], y_pred)
        else:
            recall = recall_score(y[self.attribute], y_pred, average='micro')
        return recall

In [193]:
import optuna
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
import joblib
import optuna.visualization
# from model import SVCModel

class SVCTuner:
    def __init__(self, X_train, y_train, X_val, y_val, attribute=None):
        self.X_train, self.y_train, self.X_val, self.y_val = X_train, y_train, X_val, y_val
        self.attribute = attribute
        self.best_params = None
        self.best_model = None

    def objective(self, trial):
        kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
        C = trial.suggest_float('C', 1e-5, 100)
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
        
        svc_model = SVCModel(kernel=kernel, C=C, gamma=gamma, attribute=self.attribute)
        svc_model.fit(self.X_train, self.y_train)
        
        acc = svc_model.calculate_accuracy_score(self.X_val, self.y_val)
        
        return acc

    def tune(self, n_trials=100):
        study = optuna.create_study(direction='maximize')
        study.optimize(self.objective, n_trials=n_trials)
        
        self.best_params = study.best_params
        self.best_model = SVCModel(kernel=self.best_params['kernel'], 
                                   C=self.best_params['C'], 
                                   gamma=self.best_params['gamma'], 
                                   attribute=self.attribute)
        
        self.best_model.fit(self.X_train, self.y_train)
        joblib.dump(self.best_model, 'bestModel'+self.attribute)
        
        return optuna.visualization.plot_optimization_history(study)
    
    def evaluate_best_model(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test
        if self.best_model is None:
            raise ValueError("No best model available. Please run tuning first.")
        
        base_model = SVCModel(attribute=self.attribute)
        base_model.fit(self.X_train, self.y_train)
        y_pred_base = base_model.predict(self.X_test)
        base_acc = base_model.calculate_accuracy_score(self.X_test, self.y_test, y_pred=y_pred_base)
        base_f1 = base_model.calculate_f1_score(self.X_test, self.y_test, y_pred=y_pred_base)
        base_precision = base_model.calculate_precision_score(self.X_test, self.y_test, y_pred=y_pred_base)
        base_recall = base_model.calculate_recall_score(self.X_test, self.y_test, y_pred=y_pred_base)
        
        y_pred_best = self.best_model.predict(self.X_test)
        best_acc = self.best_model.calculate_accuracy_score(self.X_test, self.y_test, y_pred=y_pred_best)
        best_f1 = self.best_model.calculate_f1_score(self.X_test, self.y_test, y_pred=y_pred_best)
        best_recall = self.best_model.calculate_recall_score(self.X_test, self.y_test, y_pred=y_pred_best)
        best_precision = self.best_model.calculate_precision_score(self.X_test, self.y_test, y_pred=y_pred_best)

        return {'Base':{'Accuracy': base_acc,
                        'Precision': base_precision,
                        'Recall': base_recall,
                        'F1': base_f1},
                'Best':{'Accuracy': best_acc,
                        'Precision': best_precision,
                        'Recall': best_recall,
                        'F1': best_f1}}

In [198]:
svm_Pin = SVCTuner(X_train, y_train, X_val, y_val, attribute='Pin')
svm_Pin.tune(n_trials=100)

[I 2024-03-31 10:36:19,602] A new study created in memory with name: no-name-b78149be-046c-479c-9d47-eeaa1c1e9d08
[I 2024-03-31 10:36:20,340] Trial 0 finished with value: 0.921875 and parameters: {'kernel': 'rbf', 'C': 69.64620040532274, 'gamma': 'scale'}. Best is trial 0 with value: 0.921875.
[I 2024-03-31 10:36:21,180] Trial 1 finished with value: 0.815625 and parameters: {'kernel': 'sigmoid', 'C': 7.1126058530794785, 'gamma': 'scale'}. Best is trial 0 with value: 0.921875.
[I 2024-03-31 10:36:21,719] Trial 2 finished with value: 0.90625 and parameters: {'kernel': 'linear', 'C': 57.01757268769069, 'gamma': 'auto'}. Best is trial 0 with value: 0.921875.
[I 2024-03-31 10:36:22,321] Trial 3 finished with value: 0.909375 and parameters: {'kernel': 'poly', 'C': 32.17120204936312, 'gamma': 'scale'}. Best is trial 0 with value: 0.921875.
[I 2024-03-31 10:36:22,910] Trial 4 finished with value: 0.7625 and parameters: {'kernel': 'sigmoid', 'C': 34.076939517716326, 'gamma': 'scale'}. Best is t

In [199]:
svm_Service = SVCTuner(X_train, y_train, X_val, y_val, attribute='Service')
svm_Service.tune(n_trials=100)

[I 2024-03-31 10:37:36,771] A new study created in memory with name: no-name-269f6023-d5d5-4c36-8fab-30c09e291cef
[I 2024-03-31 10:37:37,541] Trial 0 finished with value: 0.725 and parameters: {'kernel': 'poly', 'C': 33.02846812213452, 'gamma': 'auto'}. Best is trial 0 with value: 0.725.
[I 2024-03-31 10:37:38,187] Trial 1 finished with value: 0.715625 and parameters: {'kernel': 'sigmoid', 'C': 80.17440449433758, 'gamma': 'scale'}. Best is trial 0 with value: 0.725.
[I 2024-03-31 10:37:38,947] Trial 2 finished with value: 0.725 and parameters: {'kernel': 'poly', 'C': 7.210969060895121, 'gamma': 'auto'}. Best is trial 0 with value: 0.725.
[I 2024-03-31 10:37:39,726] Trial 3 finished with value: 0.88125 and parameters: {'kernel': 'poly', 'C': 12.107773111389237, 'gamma': 'scale'}. Best is trial 3 with value: 0.88125.
[I 2024-03-31 10:37:40,563] Trial 4 finished with value: 0.865625 and parameters: {'kernel': 'poly', 'C': 30.31525406850897, 'gamma': 'scale'}. Best is trial 3 with value: 0

In [200]:
svm_General = SVCTuner(X_train, y_train, X_val, y_val, attribute='General')
svm_General.tune(n_trials=100)

[I 2024-03-31 10:39:08,548] A new study created in memory with name: no-name-6d45822f-0a92-4943-aceb-4bbf25c0a4ba
[I 2024-03-31 10:39:09,887] Trial 0 finished with value: 0.546875 and parameters: {'kernel': 'sigmoid', 'C': 37.06966977570647, 'gamma': 'auto'}. Best is trial 0 with value: 0.546875.
[I 2024-03-31 10:39:11,251] Trial 1 finished with value: 0.709375 and parameters: {'kernel': 'poly', 'C': 54.541086397682946, 'gamma': 'scale'}. Best is trial 1 with value: 0.709375.
[I 2024-03-31 10:39:12,632] Trial 2 finished with value: 0.546875 and parameters: {'kernel': 'sigmoid', 'C': 20.63528183659958, 'gamma': 'auto'}. Best is trial 1 with value: 0.709375.
[I 2024-03-31 10:39:13,607] Trial 3 finished with value: 0.69375 and parameters: {'kernel': 'linear', 'C': 85.93676658624652, 'gamma': 'auto'}. Best is trial 1 with value: 0.709375.
[I 2024-03-31 10:39:14,987] Trial 4 finished with value: 0.546875 and parameters: {'kernel': 'sigmoid', 'C': 27.00023827628494, 'gamma': 'auto'}. Best is

In [201]:
svm_Others = SVCTuner(X_train, y_train, X_val, y_val, attribute='Others')
svm_Others.tune(n_trials=100)

[I 2024-03-31 10:41:35,281] A new study created in memory with name: no-name-fbdea713-c793-4deb-bcb6-c6f2a79276e6
[I 2024-03-31 10:41:36,057] Trial 0 finished with value: 0.709375 and parameters: {'kernel': 'poly', 'C': 69.31079496124961, 'gamma': 'auto'}. Best is trial 0 with value: 0.709375.
[I 2024-03-31 10:41:37,231] Trial 1 finished with value: 0.775 and parameters: {'kernel': 'rbf', 'C': 67.46942037442129, 'gamma': 'scale'}. Best is trial 1 with value: 0.775.
[I 2024-03-31 10:41:38,065] Trial 2 finished with value: 0.8125 and parameters: {'kernel': 'poly', 'C': 16.735870115709922, 'gamma': 'scale'}. Best is trial 2 with value: 0.8125.
[I 2024-03-31 10:41:38,901] Trial 3 finished with value: 0.709375 and parameters: {'kernel': 'poly', 'C': 75.85311491529097, 'gamma': 'auto'}. Best is trial 2 with value: 0.8125.
[I 2024-03-31 10:41:39,973] Trial 4 finished with value: 0.76875 and parameters: {'kernel': 'poly', 'C': 72.79139814041619, 'gamma': 'scale'}. Best is trial 2 with value: 0

In [202]:
svm_SPin = SVCTuner(X_train, y_train, X_val, y_val, attribute='SPin')
svm_SPin.tune(n_trials=100)

[I 2024-03-31 10:43:01,647] A new study created in memory with name: no-name-6f0e0a41-422d-43a5-b4b8-d138f33ea53f
[I 2024-03-31 10:43:02,494] Trial 0 finished with value: 0.66875 and parameters: {'kernel': 'sigmoid', 'C': 43.21386366741549, 'gamma': 'scale'}. Best is trial 0 with value: 0.66875.
[I 2024-03-31 10:43:03,309] Trial 1 finished with value: 0.775 and parameters: {'kernel': 'linear', 'C': 46.26124370137883, 'gamma': 'auto'}. Best is trial 1 with value: 0.775.
[I 2024-03-31 10:43:04,182] Trial 2 finished with value: 0.634375 and parameters: {'kernel': 'poly', 'C': 78.8471117809216, 'gamma': 'auto'}. Best is trial 1 with value: 0.775.
[I 2024-03-31 10:43:05,371] Trial 3 finished with value: 0.76875 and parameters: {'kernel': 'poly', 'C': 89.83249236207747, 'gamma': 'scale'}. Best is trial 1 with value: 0.775.
[I 2024-03-31 10:43:06,271] Trial 4 finished with value: 0.665625 and parameters: {'kernel': 'sigmoid', 'C': 44.57669403867617, 'gamma': 'scale'}. Best is trial 1 with val

In [203]:
svm_SSer = SVCTuner(X_train, y_train, X_val, y_val, attribute='SSer')
svm_SSer.tune(n_trials=100)

[I 2024-03-31 10:44:55,898] A new study created in memory with name: no-name-e68bb2bb-bc9f-41dc-b274-a82969bcb591
[I 2024-03-31 10:44:56,649] Trial 0 finished with value: 0.86875 and parameters: {'kernel': 'linear', 'C': 77.71542631742118, 'gamma': 'auto'}. Best is trial 0 with value: 0.86875.
[I 2024-03-31 10:44:57,301] Trial 1 finished with value: 0.753125 and parameters: {'kernel': 'poly', 'C': 12.60389539437282, 'gamma': 'auto'}. Best is trial 0 with value: 0.86875.
[I 2024-03-31 10:44:58,300] Trial 2 finished with value: 0.753125 and parameters: {'kernel': 'sigmoid', 'C': 52.453854044971266, 'gamma': 'auto'}. Best is trial 0 with value: 0.86875.
[I 2024-03-31 10:44:59,279] Trial 3 finished with value: 0.753125 and parameters: {'kernel': 'sigmoid', 'C': 46.09156051555469, 'gamma': 'auto'}. Best is trial 0 with value: 0.86875.
[I 2024-03-31 10:45:00,006] Trial 4 finished with value: 0.871875 and parameters: {'kernel': 'linear', 'C': 66.77854656938183, 'gamma': 'auto'}. Best is trial

In [204]:
svm_SGeneral = SVCTuner(X_train, y_train, X_val, y_val, attribute='SGeneral')
svm_SGeneral.tune(n_trials=100)

[I 2024-03-31 10:46:15,165] A new study created in memory with name: no-name-e4892c66-c695-4f9e-bbf1-178b5feca0ec
[I 2024-03-31 10:46:16,469] Trial 0 finished with value: 0.575 and parameters: {'kernel': 'sigmoid', 'C': 31.5817305358174, 'gamma': 'auto'}. Best is trial 0 with value: 0.575.
[I 2024-03-31 10:46:17,528] Trial 1 finished with value: 0.66875 and parameters: {'kernel': 'linear', 'C': 44.97917846073086, 'gamma': 'scale'}. Best is trial 1 with value: 0.66875.
[I 2024-03-31 10:46:18,635] Trial 2 finished with value: 0.659375 and parameters: {'kernel': 'linear', 'C': 91.48856645940197, 'gamma': 'scale'}. Best is trial 1 with value: 0.66875.
[I 2024-03-31 10:46:20,135] Trial 3 finished with value: 0.646875 and parameters: {'kernel': 'poly', 'C': 80.34541435410502, 'gamma': 'scale'}. Best is trial 1 with value: 0.66875.
[I 2024-03-31 10:46:21,410] Trial 4 finished with value: 0.575 and parameters: {'kernel': 'rbf', 'C': 2.0305732890714303, 'gamma': 'auto'}. Best is trial 1 with va

In [205]:
svm_SOth = SVCTuner(X_train, y_train, X_val, y_val, attribute='SOth')
svm_SOth.tune(n_trials=100)

[I 2024-03-31 10:48:41,064] A new study created in memory with name: no-name-602c51cf-a3ba-4170-b6df-02a3d6fc47c0
[I 2024-03-31 10:48:42,680] Trial 0 finished with value: 0.625 and parameters: {'kernel': 'rbf', 'C': 63.41287509730817, 'gamma': 'scale'}. Best is trial 0 with value: 0.625.
[I 2024-03-31 10:48:44,065] Trial 1 finished with value: 0.596875 and parameters: {'kernel': 'poly', 'C': 53.01474108703527, 'gamma': 'scale'}. Best is trial 0 with value: 0.625.
[I 2024-03-31 10:48:45,816] Trial 2 finished with value: 0.571875 and parameters: {'kernel': 'poly', 'C': 98.89661258086758, 'gamma': 'scale'}. Best is trial 0 with value: 0.625.
[I 2024-03-31 10:48:46,937] Trial 3 finished with value: 0.61875 and parameters: {'kernel': 'linear', 'C': 15.755987995399, 'gamma': 'auto'}. Best is trial 0 with value: 0.625.
[I 2024-03-31 10:48:48,613] Trial 4 finished with value: 0.471875 and parameters: {'kernel': 'sigmoid', 'C': 68.94230078154304, 'gamma': 'auto'}. Best is trial 0 with value: 0.

# Result

In [206]:
compare_table = {}
compare_table['Pin'] = svm_Pin.evaluate_best_model(X_test, y_test)
compare_table['Service'] = svm_Service.evaluate_best_model(X_test, y_test)
compare_table['General'] = svm_General.evaluate_best_model(X_test, y_test)
compare_table['Others'] = svm_Others.evaluate_best_model(X_test, y_test)

compare_table['SPin'] = svm_SPin.evaluate_best_model(X_test, y_test)
compare_table['SSer'] = svm_SSer.evaluate_best_model(X_test, y_test)
compare_table['SGeneral'] = svm_SGeneral.evaluate_best_model(X_test, y_test)
compare_table['SOthers'] = svm_SOth.evaluate_best_model(X_test, y_test)

In [207]:
compare_df = pd.DataFrame(compare_table).T
compare_df

,Base,Best
Pin,"{'Accuracy': 0.9069244604316546, 'Precision': 0.9458563535911603, 'Recall': 0.8441814595660749, 'F1': 0.8921313183949974}","{'Accuracy': 0.891636690647482, 'Precision': 0.8876629889669007, 'Recall': 0.8727810650887574, 'F1': 0.8801591248135255}"
Service,"{'Accuracy': 0.8430755395683454, 'Precision': 0.9586466165413534, 'Recall': 0.4300168634064081, 'F1': 0.5937136204889407}","{'Accuracy': 0.8327338129496403, 'Precision': 0.7073170731707317, 'Recall': 0.6357504215851602, 'F1': 0.669626998223801}"
General,"{'Accuracy': 0.5674460431654677, 'Precision': 0.8618307426597582, 'Recall': 0.3613323678493845, 'F1': 0.5091836734693878}","{'Accuracy': 0.6447841726618705, 'Precision': 0.7743732590529248, 'Recall': 0.6039102099927589, 'F1': 0.6786004882017901}"
Others,"{'Accuracy': 0.8192446043165468, 'Precision': 0.8791878172588833, 'Recall': 0.9135021097046413, 'F1': 0.8960165545783756}","{'Accuracy': 0.8030575539568345, 'Precision': 0.8992332968236583, 'Recall': 0.8660337552742616, 'F1': 0.8823213326168726}"
SPin,"{'Accuracy': 0.6569244604316546, 'Precision': 0.6569244604316546, 'Recall': 0.6569244604316546, 'F1': 0.6569244604316546}","{'Accuracy': 0.7585431654676259, 'Precision': 0.7585431654676259, 'Recall': 0.7585431654676259, 'F1': 0.7585431654676259}"
SSer,"{'Accuracy': 0.8358812949640287, 'Precision': 0.8358812949640287, 'Recall': 0.8358812949640287, 'F1': 0.8358812949640287}","{'Accuracy': 0.8376798561151079, 'Precision': 0.8376798561151079, 'Recall': 0.8376798561151079, 'F1': 0.8376798561151079}"
SGeneral,"{'Accuracy': 0.42356115107913667, 'Precision': 0.42356115107913667, 'Recall': 0.42356115107913667, 'F1': 0.42356115107913667}","{'Accuracy': 0.6025179856115108, 'Precision': 0.6025179856115108, 'Recall': 0.6025179856115108, 'F1': 0.6025179856115108}"
SOthers,"{'Accuracy': 0.5638489208633094, 'Precision': 0.5638489208633094, 'Recall': 0.5638489208633094, 'F1': 0.5638489208633094}","{'Accuracy': 0.6384892086330936, 'Precision': 0.6384892086330936, 'Recall': 0.6384892086330936, 'F1': 0.6384892086330936}"


## Several wrong predictions

In [110]:
#load model
svm_Pin = joblib.load('/kaggle/input/pretrained-svm/bestModelPin')
svm_Service = joblib.load('/kaggle/input/pretrained-svm/bestModelService')
svm_General = joblib.load('/kaggle/input/pretrained-svm/bestModelGeneral')
svm_Others = joblib.load('/kaggle/input/pretrained-svm/bestModelOthers')

svm_SPin = joblib.load('/kaggle/input/pretrained-svm/bestModelSPin')
svm_SSer = joblib.load('/kaggle/input/pretrained-svm/bestModelSSer')
svm_SGeneral = joblib.load('/kaggle/input/pretrained-svm/bestModelSGeneral')
svm_SOth = joblib.load('/kaggle/input/pretrained-svm/bestModelSOth')

In [111]:
y_pred_Pin = svm_Pin.predict(X_test)
y_pred_Service = svm_Service.predict(X_test)
y_pred_General = svm_General.predict(X_test)
y_pred_Others = svm_Others.predict(X_test)

y_pred_SPin = svm_SPin.predict(X_test)
y_pred_SSer = svm_SSer.predict(X_test)
y_pred_SGeneral = svm_SGeneral.predict(X_test)
y_pred_SOth = svm_SOth.predict(X_test)


In [112]:
pd.set_option('display.max_colwidth', None)

index_mismatch = np.where(y_pred_Pin != y_test['Pin'])[0]
Pin_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_Pin[index_mismatch],
    'Actual': y_test['Pin'].iloc[index_mismatch]
})
Pin_mismatch

,Comments,Predict,Actual
0,điện thoải ổn facebooklockhôngông cực nhanh vân tay ôkhôngông mìnhàn hình lớn pin trôìâu liên quân zalo youtube mìnhột ngày mìnhất khôngônghoảng tuy chỉ chip nhưng rôíât mìnhượt đa nhiệmình khôngônghá ổn,1,0
2,xấu đẹp gì khôngônghông biết nhưng rôíât ưng tgdđ phục vụ rôíât tuyệt vời mìnhua ở tgdđ mìnhòn đít và sẽ mìnhua dài dài ủng hộ tgdđ,1,0
12,nâng cấp nên phiên bản mìnhới pin sụt nhanh nghĩ mìnhà chán,1,0
14,mìnhàn hình sắc nét pin trôìâu chíp khôngônghoẻ hình thức mìnháy đẹp vivo s prôìo quá tuyệt vời,0,1
16,mìnhình mìnhua chưa được tuần mìnhà tình trôịang pin còn thì có đổi lại được hkhôngông với lại nhiều khôngônghi bị đứng mìnháy thấy nản quá,0,1
...,...,...,...
2159,mìnhikhôngông xài samìnhsung as rôíât tốt khôngônghông biz mìnhấy bn sao chứ mìnhình chs pubg với frôìe firồie liên quan khôngônghông vấn đề gì ht nha mìnhượt khôngônghông chê vào âu khôngôngo đơ khôngônghông lag mìnhikhôngông ns thật nếu mìnhấy bn bị đơ thì xin coi lại rồiamình có trôìan khôngônghông đơ là do sử dụng quá mìnhức chứ khôngônghông phải tại mìnháy chơi êmình lắmình mìnhn ạ sang trôịong tin mìnhình ikhôngông samìnhsung as tốt pin trôìâu bò mìnhình xài rôìôi êmình,0,1
2169,cảmình ứng quá khôngôngémình cho chiếc trồi khôngônghông bằng chiếc samìnhsung trồi khôngôngể cả việc nhắn tin hay chơi gamìnhe đều khôngônghông đáp ứng được độ cảmình ứng quá khôngôngémình pin tạmình được hiệu năng tốt chụp ảnh okhôngông buồn nhất mìnhỗi cảmình ứng lúc mìnhới mìnhua được đến tuần thì ngon lắmình về sau thì quá chán,0,1
2187,sản phẩmình tốt dung lượng pin lớn dùng được nhiều ngày loa nghe to rôĩo đáp ứng tốt,1,0
2198,mìnhới chưa được tháng mìnhà chay tgdđ trôỉa lời giúp ae ai cùng cảnh ngộ xin cách xử lý khôngônghông xạc qua đêmình khôngônghông vừa dùng vừa xạc nha có ai đổi được mìnháy khôngônghác khôngôngo,0,1


In [113]:
pd.set_option('display.max_colwidth', None)

index_mismatch = np.where(y_pred_Pin != y_test['Service'])[0]
Service_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_Service[index_mismatch],
    'Actual': y_test['Service'].iloc[index_mismatch]
})
Service_mismatch

,Comments,Predict,Actual
0,điện thoải ổn facebooklockhôngông cực nhanh vân tay ôkhôngông mìnhàn hình lớn pin trôìâu liên quân zalo youtube mìnhột ngày mìnhất khôngônghoảng tuy chỉ chip nhưng rôíât mìnhượt đa nhiệmình khôngônghá ổn,0,0
1,mìnhình mìnhới mìnhua vivoc tải ứng dụng gamìnhes nhanh có cái khôngônghông hài lòng là cài hình nền khôngônghóa mìnhàn hình khôngônghông được hay tại mìnhình chưa biết hết chức năng của nó tư vấn viên nhiệt tình,1,1
4,nói chung mìnháy đẹp với mìnhàn amìnholed ổn trôìong tầmình giá hơi tiếc là thời lượng pin khôngônghông cao như quảng cáo,0,0
7,sản phẩmình tốt pin xài cả ngày chưa hết chơi gamìnhe lướt web rôíât mìnhượt chụp hình đẹp cầmình vừa tay rôíât đáng đồng tiền,0,0
8,mìnhua được tuần và đt xài khôngônghá tốt giá tầmình trôìung mìnhà v là okhôngông lắmình rôìôi pin cực khôngôngì mìnhạnh điểmình yếu là camình trôìc thôi chứ camình sau đẹp lắmình nha mìnhọi người xài khôngôngĩ mìnhột tí thì mìnháy tốt lắmình chứ mìnháy gần trồi mìnhà lên đây chê này chê nọ là sao v là tốt lắmình rôìôi,0,0
...,...,...,...
2211,lần đầu nhìn mìnháy rôíât ấn tượng về thiết khôngôngế camìnherồia chụp rôíât đẹp đúng là opo chuyên camìnherồia nên tấmình nào chụp cũng đẹp pin sạc nhanh voc nên khôngônghông sợ vấn đề về pin mìnhình ghét mìnhàn hình khôngônghuyết điểmình như đục lỗ giọt nước nhưng rồieno f đã làmình tốt điều này điểmình cộng là mìnháy dùng tấmình nền amìnholed nên mìnhàu sắc rôịưc rôiỡ mìnháy tác vụ khôngônghá mìnhượt nhưng gamìnhe thì tạmình ổn nói chung mìnhình rôíât hài lòng vố sản phẩmình này,0,0
2212,mìnhua hômình hômình nay ngày mìnhọi thứ okhôngông pin cũng ngang ngửa pin mìnhấy hãng andrồioi tầmình giaáh mìnhà sao mìnhấy đứa trôỉâu chê này chê nọ,0,0
2213,nhìn trôìung về chất lượng sản phẩmình còn thua xa opo của trôìung quốc đung thì hay bị đơ thôi chót đại giờ thì cứ samìnhsung mìnhà dùng khôngônghỏi phải lo,0,1
2214,giá hợp lý cấu hình tốt nhìn chung hài lòng về sản phẩmình mìnhong vinsmìnharồit tiếp tục rôìa các sản phẩmình cao cấp hơn nữa,1,1


In [114]:
index_mismatch = np.where(y_pred_General != y_test['General'])[0]
General_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_General[index_mismatch],
    'Actual': y_test['General'].iloc[index_mismatch]
})
General_mismatch

,Comments,Predict,Actual
2,xấu đẹp gì khôngônghông biết nhưng rôíât ưng tgdđ phục vụ rôíât tuyệt vời mìnhua ở tgdđ mìnhòn đít và sẽ mìnhua dài dài ủng hộ tgdđ,1,0
4,nói chung mìnháy đẹp với mìnhàn amìnholed ổn trôìong tầmình giá hơi tiếc là thời lượng pin khôngônghông cao như quảng cáo,1,0
5,đt có lỗi là chế dô bỏ túi là ánh sáng điện thoại là nhấp đúp vào mìnhàn hình thì khôngônghông nhạy cho lắmình còn lại thì rôíât tốt,0,1
9,pin nhanh hết vân tay cũng chưa được nhạy nhân vien tgd tư vấn nhiệt tình dễ thương sao về phong cách bán hàng,1,0
12,nâng cấp nên phiên bản mìnhới pin sụt nhanh nghĩ mìnhà chán,0,1
...,...,...,...
2202,mìnháy này có chống nước khôngônghông vậy đã mìnhua ở tgd xài cũng okhôngông vừa tầmình giá này về sạc thế nào cho pin khôngônghoẻ vậy cửa hàng mìnháy bóc temình luôn,0,1
2203,mìnháy dùng okhôngông chê mìnhỗi cái loa ngoài âmình thanh hơi chán mìnhong nsx lưu ý khôngônghắc phục ở những phiên bản mìnháy sau,0,1
2206,mìnháy đã sử dụng mìnhà pin rôíât trôìâu tác vụ nhanh mìnhượt khôngônghông có gì để chê,1,0
2213,nhìn trôìung về chất lượng sản phẩmình còn thua xa opo của trôìung quốc đung thì hay bị đơ thôi chót đại giờ thì cứ samìnhsung mìnhà dùng khôngônghỏi phải lo,0,1


In [115]:
index_mismatch = np.where(y_pred_Others != y_test['Others'])[0]
Others_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_Others[index_mismatch],
    'Actual': y_test['Others'].iloc[index_mismatch]
})
Others_mismatch

,Comments,Predict,Actual
11,mìnháy thiết khôngôngế quá đẹp dùng được mìnhấy hômình rôìôi thấy mìnháy vẫn ổn tiếc là pin tụt quá nhanh pin sạc thì nhanh nóng,0,1
12,nâng cấp nên phiên bản mìnhới pin sụt nhanh nghĩ mìnhà chán,0,1
13,đang dùng tháng okhôngông sang tháng thư bắt đau lỗi chán quá săp nguồn chết khôngônghông lên được lại còn cứ khôngôngêu tắt tình ắng chế tài nghe gì ý,0,1
16,mìnhình mìnhua chưa được tuần mìnhà tình trôịang pin còn thì có đổi lại được hkhôngông với lại nhiều khôngônghi bị đứng mìnháy thấy nản quá,0,1
22,mìnháy cũng xài tạmình ổn mìnhàn hình nó khôngônghông được nhay nhiều lúc phải bấmình vài lần mìnhới được,1,0
...,...,...,...
2198,mìnhới chưa được tháng mìnhà chay tgdđ trôỉa lời giúp ae ai cùng cảnh ngộ xin cách xử lý khôngônghông xạc qua đêmình khôngônghông vừa dùng vừa xạc nha có ai đổi được mìnháy khôngônghác khôngôngo,1,0
2202,mìnháy này có chống nước khôngônghông vậy đã mìnhua ở tgd xài cũng okhôngông vừa tầmình giá này về sạc thế nào cho pin khôngônghoẻ vậy cửa hàng mìnháy bóc temình luôn,0,1
2205,mìnháy mìnhới mìnhua về xài rôíât ổn định nhân viên rôíât nhiệt tình khôngônghách hàng thân thiết được giảmình thêmình giá mìnháy nữa,1,0
2206,mìnháy đã sử dụng mìnhà pin rôíât trôìâu tác vụ nhanh mìnhượt khôngônghông có gì để chê,0,1


In [116]:
index_mismatch = np.where(y_pred_SPin != y_test['SPin'])[0]
SPin_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_SPin[index_mismatch],
    'Actual': y_test['SPin'].iloc[index_mismatch]
})
SPin_mismatch

,Comments,Predict,Actual
0,điện thoải ổn facebooklockhôngông cực nhanh vân tay ôkhôngông mìnhàn hình lớn pin trôìâu liên quân zalo youtube mìnhột ngày mìnhất khôngônghoảng tuy chỉ chip nhưng rôíât mìnhượt đa nhiệmình khôngônghá ổn,1.0,0
2,xấu đẹp gì khôngônghông biết nhưng rôíât ưng tgdđ phục vụ rôíât tuyệt vời mìnhua ở tgdđ mìnhòn đít và sẽ mìnhua dài dài ủng hộ tgdđ,1.0,0
4,nói chung mìnháy đẹp với mìnhàn amìnholed ổn trôìong tầmình giá hơi tiếc là thời lượng pin khôngônghông cao như quảng cáo,1.0,-1
8,mìnhua được tuần và đt xài khôngônghá tốt giá tầmình trôìung mìnhà v là okhôngông lắmình rôìôi pin cực khôngôngì mìnhạnh điểmình yếu là camình trôìc thôi chứ camình sau đẹp lắmình nha mìnhọi người xài khôngôngĩ mìnhột tí thì mìnháy tốt lắmình chứ mìnháy gần trồi mìnhà lên đây chê này chê nọ là sao v là tốt lắmình rôìôi,0.0,1
9,pin nhanh hết vân tay cũng chưa được nhạy nhân vien tgd tư vấn nhiệt tình dễ thương sao về phong cách bán hàng,1.0,-1
...,...,...,...
2187,sản phẩmình tốt dung lượng pin lớn dùng được nhiều ngày loa nghe to rôĩo đáp ứng tốt,1.0,0
2201,thua về mìnhọi mìnhặc trôìong tầmình giá mìnhua được tháng đơ lag camình đã yếu chơi gamìnhe yếu chỉ được cái pin okhôngông,-1.0,1
2206,mìnháy đã sử dụng mìnhà pin rôíât trôìâu tác vụ nhanh mìnhượt khôngônghông có gì để chê,-1.0,1
2212,mìnhua hômình hômình nay ngày mìnhọi thứ okhôngông pin cũng ngang ngửa pin mìnhấy hãng andrồioi tầmình giaáh mìnhà sao mìnhấy đứa trôỉâu chê này chê nọ,-1.0,1


In [117]:
index_mismatch = np.where(y_pred_SSer != y_test['SSer'])[0]
SSer_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_SSer[index_mismatch],
    'Actual': y_test['SSer'].iloc[index_mismatch]
})
SSer_mismatch

,Comments,Predict,Actual
4,nói chung mìnháy đẹp với mìnhàn amìnholed ổn trôìong tầmình giá hơi tiếc là thời lượng pin khôngônghông cao như quảng cáo,1.0,0
6,thật tuyệt mìnháy qua mìnhượt túi thích như thế giới di động có chỗ giá khôngônghác túi khôngônghông hiểu đánh giá sao cho thế giới di động phương phú châu sao nhân viên okhôngông,1.0,0
31,sản phẩmình tốt trôìong tầmình giá nhưng đôi khôngônghi mìnháy hơi bị giật lác khôngônghông được mìnhượt mìnhà lắmình,1.0,0
35,mìnhình thấy các bạn cứ chê mìnhình thấy mìnháy dùng okhôngông mìnhà khôngônghông bị lag facebookbookhôngông tikhôngôngtokhôngông coi phimình loa đều okhôngông cả mìnhình dùng điện thoại để quay phimình chụp ảnh bán hàng online thôi chứ khôngônghông gamìnhe gủng gì dùng mìnhesegerồi trôỉa lời nhắn tin khôngônghông bị đơ là sung sướng nghe gọi loa to rôĩo rồiàng nhất là khôngônghoản thu cũ đổi mìnhới còn hỗ trôiợ trôỉa góp o nữa mìnhình đổi khôngônghônge lấy mìnháy này lo khôngônghônge nhỏ mìnháy dùng hơn nămình dãi nắng dầmình mìnhưa cùng mìnhình rôìôi nên giờ cũng hay đơ nên dạo này hơi strôìes lăn tăn mìnhãi chưa đổi vì bỏ rôìa nhiều tiền để mìnhua mìnháy mìnhới quá tiếc khôngônghông nghĩ lại đơn giản đến thế luôn đưa mìnháy cũ đến cầmình mìnháy mìnhới về bắt đầu từ tháng sau mìnhỗi tháng thanh toán trồi trôìong vòng tháng thích bạn nhân viên tư vấn nhiệt tình cực khôngôngỳ,0.0,1
40,mìnháy lag khôngônghông nhúc nhích được tgd thì khôngônghông cho đổi trôỉa đã mìnhua rôíât nhiều ở tgd và đây là lần cuối cùng thề tẩy chay tgd,0.0,-1
...,...,...,...
2158,mìnhua được ngày hay bị đơ và trôìeo mìnhàn hình phải rồieset mìnháy khôngônghông biết có nên đổi mìnháy khôngônghác khôngôngo còn lại okhôngông nhân viên nhiệt tình,-1.0,1
2161,sản phẩmình rôíât okhôngông mìnhượt thiết khôngôngế đẹp mìnhắt chất lưong sản phẩmình rôíât okhôngông hàng chính hãng giá cả hợp lý cho mìnhột sản phẩmình như vâyh,1.0,0
2177,mìnháy xài được tuần pin thì sai trôìung bình xiaomìnhi pin về sao mìnhới lâu cấu hình chơi herồio surồivival người comìnhbo khôngônghông lag là okhôngông camìnherồia như cậy thì được rôìôi đối với mìnhình mìnhình thấy giá mìnháy như vậy là okhôngông lun nhân viên ở bạc liêu thì nhiệt tình và hết mìnhình hổ trôiợ cho khôngônghách mìnhình cảmình thấy vui khôngônghi mìnhua đồ ở thế giới di động,0.0,1
2203,mìnháy dùng okhôngông chê mìnhỗi cái loa ngoài âmình thanh hơi chán mìnhong nsx lưu ý khôngônghắc phục ở những phiên bản mìnháy sau,1.0,0


In [118]:
index_mismatch = np.where(y_pred_SGeneral != y_test['SGeneral'])[0]
SGeneral_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_SGeneral[index_mismatch],
    'Actual': y_test['SGeneral'].iloc[index_mismatch]
})
SGeneral_mismatch

,Comments,Predict,Actual
2,xấu đẹp gì khôngônghông biết nhưng rôíât ưng tgdđ phục vụ rôíât tuyệt vời mìnhua ở tgdđ mìnhòn đít và sẽ mìnhua dài dài ủng hộ tgdđ,1,0
4,nói chung mìnháy đẹp với mìnhàn amìnholed ổn trôìong tầmình giá hơi tiếc là thời lượng pin khôngônghông cao như quảng cáo,1,0
5,đt có lỗi là chế dô bỏ túi là ánh sáng điện thoại là nhấp đúp vào mìnhàn hình thì khôngônghông nhạy cho lắmình còn lại thì rôíât tốt,0,1
10,mìnhới xài để cặp đã nứt mìnhàn khôngônghông lên nứt nên khôngônghông có bảo hành thay mìnhàn gần bằng mìnháy mìnhới chán,-1,0
12,nâng cấp nên phiên bản mìnhới pin sụt nhanh nghĩ mìnhà chán,0,-1
...,...,...,...
2209,theo trôỉai nghiệmình của tôi thì tầmình giá này thì mìnhọi thứ khôngônghá là okhôngông nhưng chỉ có chút vấn đề là sạc pin rôíât là nóng tôi vừa mìnhới mìnhua được vài giờ đồng hồ,0,1
2212,mìnhua hômình hômình nay ngày mìnhọi thứ okhôngông pin cũng ngang ngửa pin mìnhấy hãng andrồioi tầmình giaáh mìnhà sao mìnhấy đứa trôỉâu chê này chê nọ,-1,1
2213,nhìn trôìung về chất lượng sản phẩmình còn thua xa opo của trôìung quốc đung thì hay bị đơ thôi chót đại giờ thì cứ samìnhsung mìnhà dùng khôngônghỏi phải lo,0,-1
2221,sản phẩmình tốt ai chơi gamìnhe cứ mìnhang emình về mìnhà chơi nhé ngắn gọn thế thôu,0,1


In [119]:
index_mismatch = np.where(y_pred_SOth != y_test['SOth'])[0]
SOth_mismatch = pd.DataFrame({
    'Comments': test.iloc[index_mismatch]['Comments'],
    'Predict': y_pred_SOth[index_mismatch],
    'Actual': y_test['SOth'].iloc[index_mismatch]
})
SOth_mismatch

,Comments,Predict,Actual
1,mìnhình mìnhới mìnhua vivoc tải ứng dụng gamìnhes nhanh có cái khôngônghông hài lòng là cài hình nền khôngônghóa mìnhàn hình khôngônghông được hay tại mìnhình chưa biết hết chức năng của nó tư vấn viên nhiệt tình,-1.0,1
2,xấu đẹp gì khôngônghông biết nhưng rôíât ưng tgdđ phục vụ rôíât tuyệt vời mìnhua ở tgdđ mìnhòn đít và sẽ mìnhua dài dài ủng hộ tgdđ,1.0,0
6,thật tuyệt mìnháy qua mìnhượt túi thích như thế giới di động có chỗ giá khôngônghác túi khôngônghông hiểu đánh giá sao cho thế giới di động phương phú châu sao nhân viên okhôngông,1.0,0
8,mìnhua được tuần và đt xài khôngônghá tốt giá tầmình trôìung mìnhà v là okhôngông lắmình rôìôi pin cực khôngôngì mìnhạnh điểmình yếu là camình trôìc thôi chứ camình sau đẹp lắmình nha mìnhọi người xài khôngôngĩ mìnhột tí thì mìnháy tốt lắmình chứ mìnháy gần trồi mìnhà lên đây chê này chê nọ là sao v là tốt lắmình rôìôi,1.0,0
11,mìnháy thiết khôngôngế quá đẹp dùng được mìnhấy hômình rôìôi thấy mìnháy vẫn ổn tiếc là pin tụt quá nhanh pin sạc thì nhanh nóng,0.0,1
...,...,...,...
2205,mìnháy mìnhới mìnhua về xài rôíât ổn định nhân viên rôíât nhiệt tình khôngônghách hàng thân thiết được giảmình thêmình giá mìnháy nữa,1.0,0
2206,mìnháy đã sử dụng mìnhà pin rôíât trôìâu tác vụ nhanh mìnhượt khôngônghông có gì để chê,0.0,1
2215,mìnháy khôngônghá là ngon trôìong tầmình giá này nhưng mìnháy nên được sử dụng con chip khôngônghác mìnhạnh hơn mìnhột chút để chơi gamìnhe tốt hơn,1.0,0
2217,sao mìnhình thấy mìnháy ảnh cứ vỡ khôngôngiểu nhòe rôìôi vàng vàng ai bị vậy khôngônghông hay đt mìnhình bị sao nhỉ,0.0,-1


# Demo

In [10]:
# from data_loader import clean_n_tokenize_data
# from vectorizer import W2VLoader
import gradio as gr
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import joblib

#load model
svm_Pin = joblib.load('/kaggle/input/pretrained-svm/bestModelPin')
svm_Service = joblib.load('/kaggle/input/pretrained-svm/bestModelService')
svm_General = joblib.load('/kaggle/input/pretrained-svm/bestModelGeneral')
svm_Others = joblib.load('/kaggle/input/pretrained-svm/bestModelOthers')

svm_SPin = joblib.load('/kaggle/input/pretrained-svm/bestModelSPin')
svm_SSer = joblib.load('/kaggle/input/pretrained-svm/bestModelSSer')
svm_SGeneral = joblib.load('/kaggle/input/pretrained-svm/bestModelSGeneral')
svm_SOth = joblib.load('/kaggle/input/pretrained-svm/bestModelSOth')

# w2v_300dims = W2VLoader(
#     w2v_path='/kaggle/input/phow2v/word2vec_vi_words_300dims/word2vec_vi_words_300dims.txt'
# )

#demo
aspects = ['Pin', 'Service', 'General', 'Others']
aspects_ratio = []

sentiments = ['Positive', 'Negative', 'Neutral']
sentiments_ratio = []

def sentiment_analysis(sent):
    token_sent = clean_n_tokenize_data(sent, w2v_300dims).reshape(1, -1)

    pin_prob = svm_Pin.predict_proba(token_sent).ravel()
    service_prob = svm_Service.predict_proba(token_sent).ravel()
    general_prob = svm_General.predict_proba(token_sent).ravel()
    others_prob = svm_Others.predict_proba(token_sent).ravel()

    spin_prob = svm_SPin.predict_proba(token_sent).ravel()
    sser_prob = svm_SSer.predict_proba(token_sent).ravel()
    sgen_prob = svm_SGeneral.predict_proba(token_sent).ravel()
    soth_prob = svm_SOth.predict_proba(token_sent).ravel()

    aspects_ratio.clear()
    sentiments_ratio.clear()
    aspects_ratio.extend([pin_prob, service_prob, general_prob, others_prob])
    sentiments_ratio.extend([spin_prob, sser_prob, sgen_prob, soth_prob])

def draw_bar_aspect():
    # Tạo list chứa các thanh bar
    bars = []

    # Vẽ từng thanh bar cho mỗi khía cạnh
    for i in range(len(aspects)):
        bars.append(go.Bar(
            y=[aspects[i]],
            x=[aspects_ratio[i][0]],
            orientation='h',
            name='Không',
            showlegend=True if i == 0 else False,
            marker=dict(color='orange')
        ))
        bars.append(go.Bar(
            y=[aspects[i]],
            x=[aspects_ratio[i][1]],
            orientation='h',
            name='Có',
            showlegend=True if i == 0 else False,
            marker=dict(color='blue')
        ))

    # Tạo layout
    layout = go.Layout(
        title='Aspects',
        barmode='stack',
        xaxis=dict(title='Ratio'),
        yaxis=dict(title='Aspects')
    )

    # Tạo figure
    fig = go.Figure(data=bars, layout=layout)

    return fig

def draw_bar_sentiment():
    bars = []

    # Vẽ từng thanh bar cảm xúc cho mỗi khía cạnh
    for i in range(len(aspects)):
        bars.append(go.Bar(
            y=[aspects[i]],
            x=[sentiments_ratio[i][0]],
            orientation='h',
            name='Negative',
            showlegend=True if i == 0 else False,
            marker=dict(color='orange')
        ))
        bars.append(go.Bar(
            y=[aspects[i]],
            x=[sentiments_ratio[i][1]],
            orientation='h',
            name='Neutral',
            showlegend=True if i == 0 else False,
            marker=dict(color='blue')
        ))
        bars.append(go.Bar(
            y=[aspects[i]],
            x=[sentiments_ratio[i][2]],
            orientation='h',
            name='Positive',
            showlegend=True if i == 0 else False,
            marker=dict(color='red')
        ))

    # Tạo layout
    layout = go.Layout(
        title='Sentiments of Aspects',
        barmode='stack',
        xaxis=dict(title='Ratio'),
        yaxis=dict(title='Aspects')
    )

    # Tạo figure
    fig = go.Figure(data=bars, layout=layout)

    return fig

def draw_pie_aspect():
    # Tạo subplot với loại 'pie' và khoảng trống
    fig = make_subplots(rows=1, cols=4, subplot_titles=aspects, horizontal_spacing=0.05, specs=[[{'type':'pie'}]*4])

    # Vẽ biểu đồ pie cho mỗi khía cạnh
    for i, aspect in enumerate(aspects):
        labels = ['Không', 'Có']
        values = aspects_ratio[i]
        
        # Tạo biểu đồ pie và thêm vào subplot tương ứng
        fig.add_trace(go.Pie(labels=labels, values=values, name=aspect), row=1, col=i+1)

    # Cài đặt layout
    fig.update_layout(
        title='Aspect',
    )
    return fig

def draw_pie_sentiment():
    # Tạo subplot với loại 'pie' và khoảng trống
    fig = make_subplots(rows=1, cols=4, subplot_titles=aspects, horizontal_spacing=0.05, specs=[[{'type':'pie'}]*4])

    # Vẽ biểu đồ pie cho mỗi khía cạnh
    for i, aspect in enumerate(aspects):
        labels = sentiments
        values = sentiments_ratio[i]
        
        # Tạo biểu đồ pie và thêm vào subplot tương ứng
        fig.add_trace(go.Pie(labels=labels, values=values, name=aspect), row=1, col=i+1)

    # Cài đặt layout
    fig.update_layout(
        title='Sentiment Analysis by Aspect',
    )
    return fig

def draw_plot_aspect(type_plot):
    if type_plot == 'Bar':
        return draw_bar_aspect()
    else:
        return draw_pie_aspect()
    
def draw_plot_sentiment(type_plot):
    if type_plot == 'Bar':
        return draw_bar_sentiment()
    else:
        return draw_pie_sentiment()

def draw_plot(type_aspect_plot, type_sentiment_plot):
    return draw_plot_aspect(type_aspect_plot), draw_plot_sentiment(type_sentiment_plot)

def submit(comment, type_aspect_plot, type_sentiment_plot):
    sentiment_analysis(comment)
    plot = draw_plot(type_aspect_plot, type_sentiment_plot)
    return {
        output_plot1: gr.Column(visible=True),
        plot_aspect: plot[0],
        plot_sentiment: plot[1],
    }
    

with gr.Blocks() as demo:
    with gr.Row():
        text_box = gr.Textbox(placeholder="Write your comment here...", visible=True, label="Comment")
        submit_btn = gr.Button(value="Submit")
    with gr.Row(visible=False) as output_plot1:
        with gr.Column():
            choose_plot_aspect_dropdown = gr.Dropdown(
                    choices=['Bar', 'Pie'], 
                    label="Choose Plot Type",
                    value='Bar',
                    )
            plot_aspect = gr.Plot()
        with gr.Column():
            choose_plot_sentiment_dropdown = gr.Dropdown(
                    choices=['Bar', 'Pie'], 
                    label="Choose Plot Type",
                    value='Pie',
                    )
            plot_sentiment = gr.Plot() 

    choose_plot_aspect_dropdown.select(
            fn = draw_plot_aspect,
            inputs = [choose_plot_aspect_dropdown],
            outputs = [plot_aspect],
        )
    choose_plot_sentiment_dropdown.select(
            fn = draw_plot_sentiment,
            inputs = [choose_plot_sentiment_dropdown],
            outputs = [plot_sentiment],
        )

    submit_btn.click(
    fn = submit,
    inputs = [text_box, choose_plot_aspect_dropdown, choose_plot_sentiment_dropdown],
    outputs = [output_plot1, plot_aspect, plot_sentiment],
)

demo.launch(share=False)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
